In [33]:
import os
import numpy as np
import pandas as pd

For els2002 institution selectivity classification; since F3PSSELECT variable (representing institutional selectivity classification) is based on the IPEDS 2010 Carnegie Classification: Undergraduate Profile, CCUGPROF variable, used the IPEDS 2010 standardized test score data for admissions for each college in the `MERGED2010_11_PP.csv` file.

Similarly for the hsls2009 institution selectivity classification, which used the IPEDS 2016 Carnegie Classification: Undergraduate Profile, so used the IPEDS 2016 standardized test score data for admissions for each college in the `MERGED2016_17_PP.csv` file.

In [34]:
scores_2012 = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/raw_data/SAT_ACT_data/MERGED2010_11_PP.csv', delimiter=',', 
                          encoding='cp1252', low_memory=False)
scores_2016 = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/raw_data/SAT_ACT_data/MERGED2016_17_PP.csv', delimiter=',', 
                          encoding='cp1252', low_memory=False)
most_recent_college = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/raw_data/SAT_ACT_data/Most-Recent-Cohorts-Institution.csv', 
                                  delimiter=',', encoding='cp1252', low_memory=False)


college_2002 = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/working_data/collegerecs/collegerec_els2002.csv')
college_2009 = pd.read_csv('C:/Users/athen/Downloads/Capstone/data/working_data/collegerecs/collegerec_hsls2009.csv')

In [35]:
score_columns = ['UNITID','INSTNM','SATVR25','SATVR75','SATMT25','SATMT75',
                 'SATVRMID','SATMTMID','ACTEN25','ACTEN75','ACTMT25','ACTMT75','ACTENMID','ACTMTMID']

scores_2012 = scores_2012[score_columns]
scores_2016 = scores_2016[score_columns]

In [36]:
scores_2012.shape

(7470, 14)

In [37]:
scores_2016.shape

(7238, 14)

In [38]:
scores_2012 = scores_2012.dropna(subset=['SATVR25','SATVR75','SATMT25','SATMT75',
                 'SATVRMID','SATMTMID','ACTEN25',
                 'ACTEN75','ACTMT25','ACTMT75','ACTENMID','ACTMTMID'], how='all')

scores_2016 = scores_2016.dropna(subset=['SATVR25','SATVR75','SATMT25','SATMT75',
                 'SATVRMID','SATMTMID','ACTEN25',
                 'ACTEN75','ACTMT25','ACTMT75','ACTENMID','ACTMTMID'], how='all')

In [39]:
college_els2002 = pd.merge(college_2002, scores_2012, on='UNITID')
college_hsls2009 = pd.merge(college_2009, scores_2016, on='UNITID')

In [40]:
college_els2002.shape

(1434, 24)

In [41]:
college_hsls2009.shape

(1314, 24)

In [42]:
college_els2002 = college_els2002.merge(most_recent_college[['UNITID', 'INSTNM']], on='UNITID', how='left')
college_els2002['INSTNM_x'] = college_els2002['INSTNM']
college_els2002 = college_els2002.dropna(subset=['INSTNM_x']).drop(['INSTNM_y', 'INSTNM'], axis=1)
college_els2002 = college_els2002.rename(columns={'INSTNM_x': 'INSTNM'})

In [43]:
college_els2002

,UNITID,INSTNM,selectivity,type,INSTSIZE,federalFinAid?,GROFFER,urbanicity,STABBR,HBCU,...,SATMT25,SATMT75,SATVRMID,SATMTMID,ACTEN25,ACTEN75,ACTMT25,ACTMT75,ACTENMID,ACTMTMID
0,100654,Alabama A & M University,3,1,2,1,1,1,AL,1,...,370.0,470.0,425.0,420.0,15.0,20.0,15.0,18.0,18.0,17.0
1,100663,University of Alabama at Birmingham,2,1,4,1,1,1,AL,0,...,510.0,690.0,585.0,600.0,21.0,28.0,19.0,26.0,25.0,23.0
2,100706,University of Alabama in Huntsville,2,1,3,1,1,1,AL,0,...,510.0,655.0,570.0,583.0,22.0,30.0,21.0,28.0,26.0,25.0
3,100724,Alabama State University,3,1,3,1,1,1,AL,1,...,320.0,420.0,400.0,370.0,12.0,16.0,14.0,16.0,14.0,15.0
4,100751,The University of Alabama,2,1,5,1,1,1,AL,0,...,500.0,620.0,555.0,560.0,22.0,30.0,20.0,27.0,26.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,447847,Aviator College of Aeronautical Science and Te...,-1,3,1,1,2,4,FL,0,...,NaN,NaN,NaN,NaN,1.0,28.0,7.0,21.0,15.0,14.0
1430,449764,Visible Music College,-1,2,1,1,2,1,TN,0,...,330.0,600.0,460.0,465.0,19.0,24.0,16.0,23.0,22.0,20.0
1431,454184,The King's College,1,2,1,1,2,1,NY,0,...,510.0,620.0,630.0,565.0,24.0,30.0,22.0,28.0,27.0,25.0
1432,455770,Providence Christian College,2,2,1,1,2,1,CA,0,...,470.0,570.0,585.0,520.0,19.0,30.0,17.0,29.0,25.0,23.0


In [44]:
college_hsls2009 = college_hsls2009.merge(most_recent_college[['UNITID', 'INSTNM']], on='UNITID', how='left')
college_hsls2009['INSTNM_x'] = college_hsls2009['INSTNM']
college_hsls2009 = college_hsls2009.dropna(subset=['INSTNM_x']).drop(['INSTNM_y', 'INSTNM'], axis=1)
college_hsls2009 = college_hsls2009.rename(columns={'INSTNM_x': 'INSTNM'})

In [45]:
college_hsls2009

,UNITID,INSTNM,selectivity,type,INSTSIZE,federalFinAid?,GROFFER,urbanicity,STABBR,HBCU,...,SATMT25,SATMT75,SATVRMID,SATMTMID,ACTEN25,ACTEN75,ACTMT25,ACTMT75,ACTENMID,ACTMTMID
0,100654,Alabama A & M University,3,1,3,1,1,1,AL,1,...,370.0,470.0,425.0,420.0,14.0,20.0,15.0,18.0,17.0,17.0
1,100663,University of Alabama at Birmingham,2,1,4,1,1,1,AL,0,...,490.0,660.0,560.0,575.0,22.0,30.0,19.0,26.0,26.0,23.0
2,100706,University of Alabama in Huntsville,2,1,3,1,1,1,AL,0,...,540.0,680.0,590.0,610.0,24.0,33.0,23.0,29.0,29.0,26.0
3,100724,Alabama State University,3,1,3,1,1,1,AL,1,...,360.0,460.0,415.0,410.0,14.0,19.0,15.0,17.0,17.0,16.0
4,100751,The University of Alabama,1,1,5,1,1,1,AL,0,...,490.0,620.0,550.0,555.0,23.0,33.0,22.0,29.0,28.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,486901,Milligan University,-1,2,2,1,1,2,TN,0,...,500.0,570.0,525.0,535.0,22.0,28.0,20.0,26.0,25.0,23.0
1310,487092,Emory University-Oxford College,-1,2,1,1,2,2,GA,0,...,620.0,760.0,655.0,690.0,NaN,NaN,NaN,NaN,NaN,NaN
1311,487524,Husson University,-1,2,2,1,1,1,ME,0,...,430.0,540.0,480.0,485.0,16.0,23.0,17.0,24.0,20.0,21.0
1312,488305,Elim Bible Institute and College,-1,2,1,1,2,3,NY,0,...,480.0,580.0,570.0,530.0,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# Saves each resulting dataframe as separate datasets
np.savetxt('collegerec_els2002.csv', college_els2002, delimiter=',', header=','.join(list(college_els2002.columns)),  fmt='%s')

np.savetxt('collegerec_hsls2009.csv', college_hsls2009, delimiter=',', header=','.join(list(college_hsls2009.columns)), fmt='%s')